In [53]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core.storage.storage_context import StorageContext

from IPython.display import Markdown, display
import os
from pinecone import Pinecone, PodSpec
import openai
from openai import OpenAI


In [59]:
host = "https://dotslive-epx6c5s.svc.gcp-starter.pinecone.io"
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
index = pc.Index(name="dotslive",host=host)

# Load documents and build index
documents = SimpleDirectoryReader( #documents
    input_dir = './DataJobs',
    filename_as_id= True,
    input_files = None,
    # exclude = 'resume.csv',
    exclude_hidden = True, 
    errors = "ignore",
    recursive = True, # for sub folders 
    encoding = "utf-8",
    # required_exts = ['.csv'],
    # file_extractor = ['.csv', PandasCSVReader],
    
).load_data()



# Construct vector store and customize storage context
storage_context = StorageContext.from_defaults(
    vector_store=PineconeVectorStore(index)
)

# Create vectors for the joined descriptions
index = VectorStoreIndex.from_documents(
    documents,  
    storage_context=storage_context
)


Upserted vectors: 100%|██████████| 21/21 [00:01<00:00, 18.32it/s]


In [60]:
query_engine = index.as_query_engine()
response = query_engine.query("What skills are required for Data Achitects?")

In [62]:
response.response

'Skills required for Data Architects include proficiency in SQL and NoSQL, experience in data visualization tools, at least 5 years of experience in data modeling, proficiency in multiple modeling techniques, knowledge of database storage principles, familiarity with big data systems and database management systems, proven analytical and problem-solving abilities, and a degree in Computer Science or a relevant field.'

In [69]:
import gradio as gr

def query_engine_test(query, _):
    response = query_engine.query(query)
    return str(response.response)


demo = gr.ChatInterface(fn=query_engine_test, examples=[
    "what skills are requred for machine learning engineer ?",
    "can you recommend me a course ?",
    "can you recommend me a course for cooking ?",
    "how should i imporve my managment skills ?",
    "What skills are missing in my resume ?",
    "how can i improve my resume ?"
], title="Pincone access tester")
demo.launch()


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [1]:
# try Asyncronous solution 
import nest_asyncio
import time



async def main():
    start_time = time.time()
    await index.as_query_engine()
    batch = asyncio.gather(*[index.query("What did the author do growing up?") for _ in range(100)])

    
    response = index.query("What did the author do growing up?")
    print(response)
    end_time = time.time()
    print(end_time - start_time)